<font size = +4>
    Imorting Libraries

In [57]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

<h1>
    <font size = 5>
    Define a function which extracts the mfcc, chroma, and mel features from a sound file. 4 parameters are taken- the file name and three Boolean parameters for the three features.
    </font>
        </h1>        
<body>    
mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound

chroma: Pertains to the 12 different pitch classes

mel: Mel Spectrogram Frequency
</body>

In [58]:
def feature_extract(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

<font size = +3>
    let's define a dictionary holding our data set

In [59]:
# Emotions in the dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

<font size = 4>Now, let’s load the data with a function load_data() – this takes in the relative size of the test set as parameter. x and y are empty lists; we’ll use the glob() function from the glob module to get all the pathnames for the sound files in our dataset. The pattern we use for this is: “C:\Users\Saumitra23\Documents\python\py data science\machine learning\speech emotion recogniton\Actor_*\\*.wav”. This is because our dataset looks like this:

Using our emotions dictionary, this number is turned into an emotion, and our function checks whether this emotion is in our list of observed_emotions; if not, it continues to the next file. It makes a call to feature_extract and stores what is returned in ‘feature’. Then, it appends the feature to x and the emotion to y. So, the list x holds the features and y holds the emotions. We call the function train_test_split with these, the test size, and a random state value, and return that.
</font>


In [60]:
#Load the data and extract features for each sound file

def load_data(test_size=0.2):
    #count=0
    x,y=[],[]
    for file in glob.glob("C:\\Users\\Saumitra23\\Documents\\python\\py data science\\machine learning\\speech emotion recogniton\\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = feature_extract(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        #count+=1
    #print(count)    
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [61]:
X_train, X_test, Y_train, Y_test = load_data(test_size = 0.25)

In [62]:
X_train.shape, X_test.shape

((468, 180), (156, 180))

<h1>
    No of features extracted

In [63]:
print('no of features extracted are:',X_train.shape[1])

no of features extracted are: 180


<h1>
     Now, let’s initialize an MLPClassifier. This is a Multi-layer Perceptron Classifier; it optimizes the log-loss function using LBFGS or stochastic gradient descent. Unlike SVM or Naive Bayes, the MLPClassifier has an internal neural network for the purpose of classification. This is a feedforward ANN model
    

In [64]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08,
                    hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

<font size = +2>
    Train the model

In [65]:
model.fit(X_train, Y_train)

C:\Users\Saumitra23\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

<h1>
prediction

In [66]:
y_pred = model.predict(X_test)
y_pred

array(['calm', 'happy', 'happy', 'calm', 'disgust', 'disgust', 'happy',
       'calm', 'calm', 'happy', 'fearful', 'disgust', 'calm', 'fearful',
       'happy', 'calm', 'fearful', 'calm', 'disgust', 'happy', 'calm',
       'calm', 'fearful', 'fearful', 'disgust', 'disgust', 'fearful',
       'calm', 'fearful', 'fearful', 'happy', 'happy', 'calm', 'fearful',
       'happy', 'disgust', 'fearful', 'calm', 'calm', 'calm', 'fearful',
       'fearful', 'happy', 'disgust', 'calm', 'happy', 'calm', 'calm',
       'disgust', 'fearful', 'calm', 'disgust', 'calm', 'fearful',
       'disgust', 'fearful', 'happy', 'happy', 'disgust', 'calm', 'calm',
       'fearful', 'fearful', 'happy', 'calm', 'calm', 'calm', 'disgust',
       'disgust', 'happy', 'happy', 'fearful', 'fearful', 'fearful',
       'disgust', 'fearful', 'calm', 'happy', 'disgust', 'disgust',
       'fearful', 'disgust', 'calm', 'calm', 'happy', 'fearful', 'calm',
       'calm', 'disgust', 'fearful', 'calm', 'fearful', 'calm', 'disgust

<h1>
    accuracy of our model

In [67]:
print('accuracy percent of our model is',accuracy_score(y_pred, Y_test)*100,'%')

accuracy percent of our model is 77.56410256410257 %
